In [61]:
import numpy as np
from google.colab import drive
import glob
from collections import Counter
import pandas as pd
import easydict
import shutil
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import copy, pickle, os, time
import argparse

from hyperopt import hp
from hyperopt.pyll.stochastic import sample
from hyperopt import tpe
from hyperopt import Trials
from hyperopt import fmin
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer

#acces to my drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
# Define the search space
space = {
    'pretrained': hp.choice('pretrained',['resnet18','resnet50', 'googlenet', 'vgg16','squeezenet','densenet']),
    'optimizer': hp.choice('optimizer',['SGD','Adam','RMSprop','Adagrad','Adadelta','Adamax']),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'batch_size': hp.quniform('batch_size', 5, 100, 5),
    'momentum': hp.quniform('momentum', 0, 1, 0.1),
}

In [63]:
# Define some parameters and paths
args = easydict.EasyDict(
    {
    "epochs": 1,
    "num_workers": 6,
    "num_class": 2,
    "max_evals": 2,
    "original_dataset_path": '/content/drive/MyDrive/Articulo/dataset5k/originalData/',
    "selected_dataset_path": '/content/drive/MyDrive/Articulo/dataset10k/selectData/',
    "test_dataset_path": '/content/drive/MyDrive/Articulo/dataset5k/testData/',
    "path_model" : '/content/drive/MyDrive/Miguel UNSA/',
    "name_csv": 'gbm_trials_batch.csv',
    "name_hyperopt": 'my_model_batch.hyperopt',
    })

In [64]:
sample(space)

{'batch_size': 25.0,
 'learning_rate': 0.037402977848495736,
 'momentum': 1.0,
 'optimizer': 'RMSprop',
 'pretrained': 'resnet18'}

In [65]:
# Define pre-processing data
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [66]:
# Preparate data in pytorch format
def process_standard_data(batch_size):

  image_datasets = {x: datasets.ImageFolder(os.path.join(args.selected_dataset_path, x),
                                                data_transforms[x])
                        for x in ['train', 'val']}

  dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size= batch_size,
                                                  shuffle=True, num_workers= args.num_workers)
                    for x in ['train', 'val']}

  dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
  class_names = image_datasets['train'].classes  ## 0: non-covid, and 1: covid-19
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  return dataloaders, dataset_sizes, device

In [67]:
# Train model using pre-trained model
def train_model(model, criterion, optimizer, scheduler, batch_szie, num_epochs, dataloaders, dataset_sizes, device):
    since = time.time()
   
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_acc= list()
    valid_acc= list()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_prec= 0.0
            running_rec = 0.0
            running_f1  = 0.0

            # Iterate over data.
            cur_batch_ind= 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                cur_acc= torch.sum(preds == labels.data).double()/batch_szie
                cur_batch_ind +=1
                
                if phase=='train':
                    train_acc.append(cur_acc)
                else:
                    valid_acc.append(cur_acc)
                
            epoch_loss= running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]            

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_epoch= epoch
                best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc= %.3f at Epoch: %d' %(best_acc,best_epoch) )

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, best_acc

In [68]:
def set_requires_grad(model_conv):
  for param in model_conv.parameters():
    param.requires_grad = False

In [69]:
# Configure the last layer for Resnet18 model, Resnet50 model, and Googlenet model.
def configure_lastlayer_mix(model_conv):

  set_requires_grad(model_conv)
  # Parameters of newly constructed modules have requires_grad=True by default     
  num_ftrs = model_conv.fc.in_features        
  model_conv.fc = nn.Linear(num_ftrs, args.num_class)

  return model_conv, model_conv.fc

In [70]:
# Configure the last layer for vgg16 model.
def configure_lastlayer_vgg16(model_conv):

  set_requires_grad(model_conv) 
  num_ftrs = model_conv.classifier[3].in_features 
  features = list(model_conv.classifier.children())[:-1] # Remove last layer
  features.extend([nn.Linear(num_ftrs, args.num_class)]) # Add our layer with 4 outputs
  model_conv.classifier = nn.Sequential(*features) # Replace the model classifier

  return model_conv, model_conv.classifier

In [71]:
# Configure the last layer for Squeezenet model.
def configure_lastlayer_squeezenet(model_conv):

  set_requires_grad(model_conv)
  model_conv.classifier[1] = nn.Conv2d(512, args.num_class, kernel_size=(1,1), stride=(1,1))
  model_conv.num_classes = args.num_class

  return model_conv

In [72]:
# Configure the last layer for Densenet model.
def configure_lastlayer_densenet(model_conv):

  set_requires_grad(model_conv)    
  num_ftrs = model_conv.classifier.in_features        
  model_conv.classifier = nn.Linear(num_ftrs, args.num_class)

  return model_conv, model_conv.classifier

In [73]:
# Load pre-trained models and configure last layer
def return_pre_training_model(learning_rate, momentum, pretrained, optimizer, device, pretrained_bool ):

  if pretrained == 'resnet18':
    model_conv = torchvision.models.resnet18(pretrained=pretrained_bool)
    model_conv, model_fc = configure_lastlayer_mix(model_conv)

  elif pretrained == 'resnet50':
    model_conv = torchvision.models.resnet50(pretrained=pretrained_bool)
    model_conv, model_fc = configure_lastlayer_mix(model_conv)

  elif pretrained == 'googlenet':
    model_conv = torchvision.models.googlenet(pretrained=pretrained_bool)
    model_conv, model_fc = configure_lastlayer_mix(model_conv)

  elif pretrained == 'vgg16':
    model_conv = torchvision.models.vgg16(pretrained=pretrained_bool)
    model_conv, model_fc = configure_lastlayer_vgg16(model_conv)

  elif pretrained == 'squeezenet':
    model_conv = torchvision.models.squeezenet1_0(pretrained=pretrained_bool)
    model_conv = configure_lastlayer_squeezenet(model_conv)
    model_fc = model_conv

  elif pretrained == 'densenet':
    model_conv =torchvision.models.densenet121(pretrained=pretrained_bool)
    model_conv, model_fc = configure_lastlayer_densenet(model_conv)

  # Use GPU or CPU
  model_conv = model_conv.to(device)  
  criterion = nn.CrossEntropyLoss()

  # Observe that only parameters of final layer are being optimized as
  # opoosed to before.
  
  if optimizer == 'SGD':
    optimizer_conv = optim.SGD(model_fc.parameters(), lr= learning_rate, momentum= momentum)

  elif optimizer == 'Adam':
    optimizer_conv = optim.Adam(model_fc.parameters(), lr= learning_rate)

  elif optimizer == 'RMSprop':
    optimizer_conv = optim.RMSprop(model_fc.parameters(), lr= learning_rate, momentum= momentum)

  elif optimizer == 'Adagrad':
    optimizer_conv = optim.Adagrad(model_fc.parameters(), lr= learning_rate)

  elif optimizer == 'Adadelta':
    optimizer_conv = optim.Adadelta(model_fc.parameters(), lr= learning_rate)
    
  elif optimizer == 'Adamax':
    optimizer_conv = optim.Adamax(model_fc.parameters(), lr= learning_rate)

  # Decay LR by a factor of 0.1 every 7 epochs
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

  return model_conv, criterion, optimizer_conv, exp_lr_scheduler

In [74]:
# Function that train and get the best model of each iterations
def evaluation_model(model_conv_pretrain, criterion, optimizer_conv, exp_lr_scheduler, dataloaders, dataset_sizes, batch_size, device):
  
  model_conv, valid_acc = train_model(model_conv_pretrain, criterion, optimizer_conv,
                         exp_lr_scheduler, batch_size, args.epochs, dataloaders, dataset_sizes, device)
  model_conv.eval()
  #torch.save(model_conv, args.path_model+'/covid_sort2_resnet18_epoch%d.pt' %args.epochs )
  return valid_acc

In [75]:
# Defines the objective function for loss minimization
def objective(params):
    """Objective function for classifier  Hyperparameter Optimization"""

    print(params)
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1


    start = timer()
    
    dataloaders_, dataset_sizes_, device_ = process_standard_data(int(params['batch_size']))
    model_conv_pretrain, criterion, optimizer_conv, exp_lr_scheduler = return_pre_training_model(params['learning_rate'], params['momentum'], params['pretrained'], params['optimizer'], device_, True)
    best_score = evaluation_model(model_conv_pretrain, criterion, optimizer_conv, exp_lr_scheduler,  dataloaders_, dataset_sizes_, int(params['batch_size']), device_)
    best_score = float(best_score.cpu().numpy()) #cuando se usa Gpu convertir tensor a .cpu()

    run_time = timer() - start
    
    # Loss must be minimized
    loss = 1 - best_score

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, run_time])
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION, 
            'train_time': run_time, 'status': STATUS_OK}

In [76]:
# Optimization algorithm
tpe_algorithm = tpe.suggest

In [77]:
# Open or create file CVS to save values of hiperparameters
out_file = args.path_model+args.name_csv
if not os.path.exists(out_file):
  print('Create new file CSV')
  # File to save first results
  of_connection = open(out_file, 'w')
  writer = csv.writer(of_connection)
  # Write the headers to the file
  writer.writerow(['loss', 'params', 'iteration', 'train_time'])
  of_connection.close()
else:
  print('CSV already exists')

CSV already exists


In [78]:
# Try to load an already saved trials object, and increase the max
MAX_EVALS = args.max_evals
try:  
  bayes_trials = pickle.load(open(args.path_model+args.name_hyperopt, "rb"))
  print("Found saved Trials! Loading...")
  MAX_EVALS = len(bayes_trials.trials) + MAX_EVALS #+ trials_step
  print(str(len(bayes_trials.trials))+" iterations have already been run")
except:  # create a new trials object and start searching
  bayes_trials = Trials()
  print('Create new file hyperopt')

Found saved Trials! Loading...
2 iterations have already been run


In [79]:
# Execute bay optimization, using the function for minimization

#%%capture

# Global variable
global  ITERATION

ITERATION = len(bayes_trials.trials)

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(50))

print("Best:", best)
    
# save the trials object
with open(args.path_model+args.name_hyperopt, "wb") as f:
  pickle.dump(bayes_trials, f)

{'batch_size': 5.0, 'learning_rate': 0.040518777517780005, 'momentum': 0.2, 'optimizer': 'Adamax', 'pretrained': 'googlenet'}
  0%|          | 0/2 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Epoch 1/1
----------
  0%|          | 0/2 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)



Training complete in 5m 18s
Best val Acc= 0.895 at Epoch: 0
{'batch_size': 15.0, 'learning_rate': 0.016149847971048317, 'momentum': 0.2, 'optimizer': 'Adam', 'pretrained': 'resnet18'}
Epoch 1/1
----------
Training complete in 3m 49s
Best val Acc= 0.929 at Epoch: 0
100%|██████████| 2/2 [09:07<00:00, 273.93s/it, best loss: 0.0707692307692308]
Best: {'batch_size': 15.0, 'learning_rate': 0.016149847971048317, 'momentum': 0.2, 'optimizer': 1, 'pretrained': 0}


In [80]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15670998718007900824]